##### Prerequisites

##### Abstract

# Introduction

# Data and methods

In [1]:
# third party libs
import os
import re
import zipfile
import logging
import threading

import pandas as pd
import geopandas as gpd

from pathlib import Path
from IPython.display import clear_output
from bokeh.transform import jitter
from bokeh.layouts import gridplot
from bokeh.models import HoverTool, FactorRange
from bokeh.plotting import output_notebook, show, figure, ColumnDataSource

# custom libs
from src.utils import (get_data_dir,)
                    

directories = """
data
data.log
data.tmp
data.ana
data.proc
data.core
data.core.ifl
data.core.gfc
data.core.soil
data.core.gl30
data.core.esvd
data.core.biomass
data.auxiliary
data.auxiliary.masks
"""

for item in directories.split():
    path = os.sep.join(item.split('.'))
    try:
        os.mkdir(path)
    except OSError:
        pass

# init Logging
formater = logging.Formatter('%(asctime)s %(levelname)s: %(message)s', '%d/%m/%y %H:%M:%S')
handler = logging.FileHandler(str(DIRS.log / 'jupyter.log'), 'a+')
handler.setLevel(logging.DEBUG)
handler.setFormatter(formater)

LOGGER = logging.getLogger('Master')
LOGGER.setLevel(logging.DEBUG)
LOGGER.addHandler(handler)

# Convenient access to data directory, is a namedtuple with folder names as attributes
DIRS = get_data_dir(str(Path('data').resolve()))

# Many functions of the processing pipeline are multi-threaded this attribute controls
# max number of threads
THREADLIMIT = 12

SPOOF_AGENT = {'headers': {'User-Agent': "Mozilla/5.0 (X11; U; Linux i686) Gecko/20071127 Firefox/2.0.0.11"}}

WGS84 = {'init': 'epsg:4326'}

# force bokeh plot output to jupyter notebook
output_notebook()

NameError: name 'DIRS' is not defined

## Data

### Core data

#### Global Forest Change
[**Global Forest Change 2000-2012 Version 1.0**](https://earthenginepartners.appspot.com/science-2013-global-forest/download_v1.0.html) (GFC) is the first high resolution dataset that provides a comprehensive view on the annual global forest cover change between 2000 and 2012 \cite{Hansen2013, Li2017}. The initial GFC dataset released by Hansen et al. is extended by recent releases which encompass the annual forest cover changes between [2000-2013 (Version 1.1)](https://earthenginepartners.appspot.com/science-2013-global-forest/download_v1.1.html), [2000-2014 (Version 1.2)](https://earthenginepartners.appspot.com/science-2013-global-forest/download_v1.2.html), [2000-2015 (Version 1.3)](https://earthenginepartners.appspot.com/science-2013-global-forest/download_v1.3.html) and [2000-2016 (Version 1.4)](https://earthenginepartners.appspot.com/science-2013-global-forest/download_v1.4.html) respectively. All versions of this dataset has in common, that they are derived from growing season imagery captured by the remote sensing satellite Landsat 7 Enhanced Thematic Mapper Plus (ETM+) at a spatial resolution of 30 meters per pixel \cite{Hansen2013a}. On the satellite imagery a time-series spectral metrics analysis is applied to gather the global forest extent at 2000 as well as the annual forest loss and gain. Hence, GFC comprises three independent data layers  tree cover, annually forest loss and  forest gain divided into 10x10 degree tiles by the geodetic coordinate system *World Geodetic System 1984* (EPSG:4326). Furthermore, across the provided layers the pixel data is coded in unsigned 8 bit integers. Hansen et al. defined trees as all vegetation taller than 5 meters for their study. Forest loss is defined as a stand displacement disturbance leading from a forest state to a non forest-state. To compute this losses 

[Global Forest Watch](http://www.globalforestwatch.org/) interactive map

- Flow general what is gfc then detailed info monitoring method, details of the different layers, how certain is the info
- trees defined as all vegetation higher than 5 meters Hansen2013, Hansen2013a
- forest loss defined as a stand displacement disturbance (> x% crown cover to 0% crown cover)  Hansen2013, Hansen2013a
- monitored by a reference percent tree cover stratum Hansen2013, Hansen2013a
- forest degeneration for example selective removals btw. all impacts on forest which are not lead to a non forest state are not considered Hansen2013a
- term forest refer to tree cover Hansen2013a
- gain is the inverse of loss e.g. the change of a non forest state to forest (crown cover densities >50%)
- Forest loss detection is less uncertain then gain detection (loss is more reliable) Li2017
- Gain is a more gradual and ecological complex process, signal is more difficult to detect Li2017
- Li2017 compares 4 different forest cover change products on their performance to estimate loss and gain patterns in china
- at the end show a example picture of the data


\cite{Hansen2013}
\cite{Hansen2013a}
\cite{Tropek2014}
\cite{Bellot2014}
\cite{Li2017}
\cite{Li2017a}

![Hansen preview](img/hansen_preview.png)

In [ ]:
# data source URL
head = 'http://commondatastorage.googleapis.com/earthenginepartners-hansen/GFC2013/'
# files to download from source url
tails = 'treecover2000.txt gain.txt lossyear.txt'.split()

data_urls = []
for tail in tails:
    content = download(head + tail)
    data_urls += content.decode('utf-8').splitlines()

to_download = []
for url in data_urls:
    lat_lon = re.search(r'(\d{2}\w_\d{3}\w)(?=\.tif)', url).groups()[0]
    lat = orientation_to_int(lat_lon.split('_')[0])
    if -20 <= lat <= 30:
        to_download.append(url)

threads = []
for idx, url in enumerate(to_download):
    if len(threads) == THREADLIMIT:
        [thread.join() for thread in threads]
        threads = []
        clear_output()
        print('Downloaded {} of {}'.format(idx, len(to_download)))
        
    path = str(DIRS.gfc / url.split('/')[-1])
    thread = threading.Thread(target=worker, args=(url, path))
    thread.start()
    threads.append(thread)

#### GlobalLand30
[GlobLand30](http://www.globallandcover.com/GLC30Download/index.aspx) (GL30)

![Chen preview](img/chen_preview.png)

#### Aboveground live woody biomass density
[Aboveground Live Woody Biomass Density](http://data.globalforestwatch.org/datasets/8f93a6f94a414f9588ce4657a39c59ff_1) (LWBD)

In [ ]:
url = 'http://data.globalforestwatch.org/datasets/8f93a6f94a414f9588ce4657a39c59ff_1.geojson'
path = str(DIRS.masks / 'biomass.geojson')

content = download(url)
write_binary(content, path)

biomass_mask = gpd.read_file(path)
to_download = list(biomass_mask.download) + list(biomass_mask.confidence) 

threads = []
for idx, url in enumerate(to_download):
    if len(threads) == THREADLIMIT:
        [thread.join() for thread in threads]
        threads = []
        clear_output()
        print('Downloaded {} of {}'.format(idx, len(to_download)))
    
    path = str(DIRS.biomass / url.split('/')[-1])
    thread = threading.Thread(target=worker, args=(url, path), kwargs=SPOOF_AGENT)
    thread.start()
    threads.append(thread)

#### Global soil organic carbon map
[The Global Soil Organic Carbon Map](http://www.fao.org/world-soil-day/global-soil-organic-carbon-map/en/) (GSOCmap)

In [ ]:
# Download broken fix it please
url = 'https://unfao-my.sharepoint.com/personal/guillermo_olmedo_fao_org/_layouts/15/guestaccess.aspx?docid=059b0b724d08a42e4931c35cff99a15c1&authkey=AcRgiPkRQvm_kuJb-K0-e2o&e=e70096969c4e4ce084d2ee1d1ca8bc44'

content = download(url, **SPOOF_AGENT)
write_binary(content, str(DIRS.soil / 'GSOCmap.tif'))

#### Ecosystem service valuation database
[Ecosystem Service Valuation Database](https://www.es-partnership.org/services/data-knowledge-sharing/ecosystem-service-valuation-database/) (ESVD)

In [ ]:
url = 'https://www.es-partnership.org/wp-content/uploads/2016/06/ESVD-TEEB-database.xls'

content = download(url, **SPOOF_AGENT)
write_binary(content, str(DIRS.esvd / url.split('/')[-1]))

#### Intact Forest Landscapes
[Intact Forest Landscapes](http://intactforests.org/index.html) (IFL2000)

In [ ]:
url = 'http://intactforests.org/shp/IFL_2000.zip'

content = download(url, **SPOOF_AGENT)
write_binary(content, str(DIRS.ifl / url.split('/')[-1]))

zipfile.ZipFile(str(DIRS.ifl / url.split('/')[-1])).extractall(str(DIRS.ifl))
os.remove(str(DIRS.ifl / url.split('/')[-1]))

### Auxiliary data

[Natural Earth Data](http://www.naturalearthdata.com/)

In [ ]:
url = 'http://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_populated_places.zip'

content = download(url, **SPOOF_AGENT)
write_binary(content, str(DIRS.masks / url.split('/')[-1]))

zipfile.ZipFile(str(DIRS.masks / url.split('/')[-1])).extractall(str(DIRS.masks))
os.remove(str(DIRS.masks / url.split('/')[-1]))

url = 'http://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_admin_0_countries.zip'

content = download(url, **SPOOF_AGENT)
write_binary(content, str(DIRS.masks / url.split('/')[-1]))

zipfile.ZipFile(str(DIRS.masks / url.split('/')[-1])).extractall(str(DIRS.masks))
os.remove(str(DIRS.masks / url.split('/')[-1]))

## Methods

### Preprocessing

#### Masking

In [ ]:
# GFC mask
gfc = sorted(DIRS.gfc.glob('*.tif'))

data_len = int(len(gfc)/3)

kwargs = {
    'gain': [i.name for i in gfc[:data_len]],
    'loss': [i.name for i in gfc[data_len:2*data_len]],
    'cover': [i.name for i in gfc[2*data_len:]],
}

gfc_mask = tile_index(gfc[:data_len], WGS84, **kwargs)
gfc_mask.to_file(str(DIRS.masks / 'gfc_mask.shp'))
gfc_mask.head()

In [ ]:
# GL30 mask
gl30 = sorted(DIRS.gl30.glob('*.tif'), key=lambda key: (key.name[7:11], key.name[0:6]))

exclude = 'n01_00 s01_00 s01_10 s01_15 s01_20 s60_00 s60_05 s60_10 s60_15 n53_00'.split()
gl30 = [item for item in gl30 if item.name[0:6] not in exclude]
data_len = int(len(gl30)/2)

kwargs = {
    'gl30_00': [i.name for i in gl30[:data_len]],
    'gl30_10': [i.name for i in gl30[data_len:]],
    'key': [i.name[0:6] for i in gl30[:data_len]]
}

gl30_mask = tile_index(gl30[data_len:], WGS84, **kwargs)
gl30_mask.to_file(str(DIRS.masks / 'gl30_mask.shp'))
gl30_mask.head()

In [ ]:
# Biomass mask
biomass = gpd.read_file(str(DIRS.masks / 'biomass.geojson'))
biomass_mask = biomass.drop(biomass.columns[[0, 1, 4, 5]], axis=1)
biomass_mask.rename(columns={'download': 'biomass'}, inplace=True)

for idx, row in biomass_mask.iterrows():
    biomass = row.biomass.split('/')[-1]
    confidence = row.confidence.split('/')[-1]
    
    biomass_mask.at[idx, 'biomass'] = biomass
    biomass_mask.at[idx, 'confidence'] = confidence

biomass_mask.to_file(str(DIRS.masks / 'biomass_mask.shp'))
biomass_mask.head()

#### Raster alignment

In [ ]:
gl30_mask = gpd.read_file(str(DIRS.masks / 'gl30_mask.shp'))
gfc_mask = gpd.read_file(str(DIRS.masks / 'gfc_mask.shp'))
biomass_mask = gpd.read_file(str(DIRS.masks / 'biomass_mask.shp'))

intersect = gpd.overlay(gfc_mask, gl30_mask, how='intersection')
intersect = gpd.overlay(intersect, biomass_mask, how='intersection')

threads = []
for key, values in intersect.groupby(by='key', sort=False):
    if len(threads) == THREADLIMIT:
        [thread.join() for thread in threads]
        threads = []
    
    to_reproject = [
        str(DIRS.gl30 / name)
        for name in list(*zip(set(values.gl30_10), set(values.gl30_00)))
    ]
    to_reproject.append(str(DIRS.soil / 'GSOCmapV1.1.tif'))
    to_merge = [
        [str(DIRS.gfc / name) for name in set(values.cover)],
        [str(DIRS.gfc / name) for name in set(values.loss)],
        [str(DIRS.gfc / name) for name in set(values.gain)],
        [str(DIRS.biomass / name) for name in set(values.biomass)],
        [str(DIRS.biomass / name) for name in set(values.confidence)],
    ]
    generic_name = '{}.tif'.format(key)
    
    thread = threading.Thread(target=worker,
                              args=(to_reproject, WGS84, to_merge, str(DIRS.proc), generic_name))
    thread.start()
    threads.append(thread)

#### Cropping and masking

In [ ]:
files = defaultdict(list)
regex = re.compile(r'.*(?P<key>(?:n|s)\d{2}_\d{2}).*', re.I)

for path in DIRS.proc.glob('*.tif'):
    match = regex.match(str(path))
    files[match.group('key')].append(path)
    files[match.group('key')] = sorted(files[match.group('key')])

threads = []
features = []
polygons = []
length = len(files)
idx = 0
for key, values in files.items():
    if len(threads) == THREADLIMIT:
        [thread.join() for thread in threads]
        threads = []
        
        clear_output()
        ratio = round((idx / length) * 100, 2)
        print('Processed {} % of {} %'.format(ratio, 100))
    
    bounds, profile = fetch_metadata(values[0], 'bounds', 'profile')
    bounds = round_bounds(bounds)

    thread = threading.Thread(target=clip_worker, args=(values, bounds, profile, str(DIRS.proc),))
    thread.start()
    threads.append(thread)
    
    key = int_to_orient(bounds.left, bounds.top)
    feature = dict([dispatch_name('{0[0]}_{0[1]}'.format(item.name.split('_')), key, idx)
                    for idx, item in enumerate(values)])
    feature['key'] = key
    features.append(feature)
    polygons.append(polygon_from(bounds))
    
    idx += 1

geometry = gpd.GeoSeries(polygons)
df = pd.DataFrame(features)
layer = gpd.GeoDataFrame(df, geometry=geometry)
layer.crs = WGS84

layer.to_file(str(DIRS.masks / 'final_mask.shp'))

In [ ]:
countries = gpd.read_file(str(DIRS.masks / 'ne_10m_admin_0_countries.shp'))
tiles = gpd.read_file(str(DIRS.masks / 'final_mask.shp'))
tiles.crs = WGS84

aio = countries.cx[:,-23:23]
aio = aio[['REGION_UN', 'geometry']]
continents = aio.dissolve(by='REGION_UN')

layer = gpd.sjoin(tiles, continents, how='left', op='intersects')
layer.to_file(str(DIRS.masks / 'final_region_mask.shp'))

#### Class harmonization

In [ ]:
mask = gpd.read_file(str(DIRS.masks / 'final_region_mask.shp'))
q = queue.Queue()
threads = []
for idx, val in mask.iterrows():
    if len(threads) == 4:
        [thread.join() for thread in threads]
        threads = []
        clear_output()
        print('{}% of 100%'.format(((idx+1)/len(mask))*100))
    
    thread = threading.Thread(target=worker, args=(DIRS.proc/val.gl30_00, DIRS.proc/val.cover, q, val.key, val.index_righ))
    thread.start()
    threads.append(thread)

In [ ]:
vals = []
while not q.empty():
    vals.append(q.get())

df = pd.DataFrame(vals)
df.columns = ['tile', 'region', 'jc0', 'smc0', 'jc10', 'smc10', 'jc20', 'smc20', 'jc30', 'smc30']
df.to_csv(DIRS.ana/'class_harmon_wet.csv', float_format='%.4f')

### Processing

# Results

## Preprocessing

#### Class harmonization

In [ ]:
src = pd.read_csv(str(DIRS.ana / 'class_harmon_wet.csv'))

# initial data clean up
src.rename(columns=lambda x: x.upper() if x[:2] == 'jc' else x, inplace=True)
src.drop('smc0 smc10 smc20 smc30'.split(), axis=1, inplace=True)
src.drop(src.columns[0], axis=1, inplace=True)
src.dropna(axis=0, how='any', inplace=True)

# scatterplot data prep
melted = src.melt(id_vars='tile region'.split(), var_name='jc_class', value_name='score')
melted['colors'] = '#ffffff'
melted.loc[melted['jc_class'] == 'JC0', 'colors'] = '#e66101'
melted.loc[melted['jc_class'] == 'JC10', 'colors'] = '#fdb863'
melted.loc[melted['jc_class'] == 'JC20', 'colors'] = '#b2abd2'
melted.loc[melted['jc_class'] == 'JC30', 'colors'] = '#5e3c99'
melted.sort_values(by=['region', 'jc_class'], ascending=[True, True], inplace=True)

# boxplot data prep
frames = []
for key, df in src.groupby('region'):
    boxplot = df.quantile(q=(0.25, 0.5, 0.75)).T
    boxplot.columns = ['q1', 'q2', 'q3']
    boxplot['iqr'] = boxplot.q3 - boxplot.q1
    boxplot['tukey_lower_whisker'] = boxplot.q1 - 1.5 * boxplot.iqr
    boxplot['tukey_upper_whisker'] = boxplot.q3 + 1.5 * boxplot.iqr
    boxplot['q_lower_whisker'] = df.quantile(q=0.025)
    boxplot['q_upper_whisker'] = df.quantile(q=0.975)
    boxplot['min_whisker'] = df.min()
    boxplot['max_whisker'] = df.max()
    boxplot['means'] = df.mean()
    boxplot['region'] = pd.unique(df.region)[0]

    frames.append(boxplot)

box = pd.concat(frames)

In [ ]:
# titel and histogram
# scatterplot
source = ColumnDataSource({'x': list(zip(melted.region, melted.jc_class)),
                           'y': melted.score,
                           'id': melted.tile,
                           'colors': melted.colors})
hover = HoverTool(tooltips=[('Region/Class', '@x'),
                            ('Tile', '@id'),
                            ('JC-Score', '@y'),])
factors = [(reg, cls) 
           for reg in pd.unique(melted.region) 
           for cls in pd.unique(melted.jc_class)]

scatter = figure(x_range=FactorRange(*factors), plot_width=950, plot_height=600,
              tools=[hover, 'pan', 'wheel_zoom', 'save', 'reset', 'box_zoom'],
              title="Jaccard score per forest cover class")

scatter.x(x=jitter('x', width=0.6, range=scatter.x_range), y='y', color='colors', source=source)

scatter.xgrid.grid_line_color = None
scatter.xaxis.axis_label = "Region/Class"
scatter.yaxis.axis_label = "Jaccard score"
scatter.y_range.start = -0.01

# boxplot
source = ColumnDataSource({'x': list(zip(box.region, box.index)),
                           'q1': box.q1,
                           'q2': box.q2,
                           'q3': box.q3,
                           'iqr': box.iqr,
                           'lw': box.min_whisker,
                           'uw': box.max_whisker,
                           'means': box.means})
hover = HoverTool(tooltips=[("Region/Class", "@x"),
                            ("Q1", "@q1"),
                            ("Q2", "@q2"),
                            ("Q3", "@q3"),
                            ("IQR", "@iqr"),
                            ("lWhisker", "@lw"),
                            ("uWhisker", "@uw"),
                            ("Mean", "@means"),])

plot = figure(x_range=scatter.x_range, y_range=scatter.y_range,
              plot_width=950, plot_height=300,
              tools=[hover, 'pan', 'wheel_zoom', 'save', 'reset', 'box_zoom'])

# box
plot.vbar(x='x', width=0.7, bottom='q1', top='q2',
          line_color='black', fill_color='#f7f7f7', fill_alpha=0.7, source=source)
plot.vbar(x='x', width=0.7, bottom='q2', top='q3',
          line_color='black', fill_color='#67a9cf', fill_alpha=0.7, source=source)

# whiskers
plot.rect(x='x', y='lw', width=0.2, height=0.001,
          line_color="black", source=source)
plot.rect(x='x', y='uw', width=0.2, height=0.001,
          line_color="black", source=source)

# stems
plot.segment(x0='x', y0='lw', x1='x', y1='q1',
             line_color='black', source=source)
plot.segment(x0='x', y0='q3', x1='x', y1='uw',
             color='black', source=source)

# mean cross
plot.x(x='x', y='means', color='#ef8a62', size=10, source=source)

plot.xgrid.grid_line_color = None
plot.xaxis.axis_label = "Region/Class"
plot.yaxis.axis_label = "Jaccard score"
plot.y_range.start = -0.01

# display plots
show(gridplot([[scatter],[plot]]))

# Discussion

# References

[<a id="cit-Hansen2013" href="#call-Hansen2013">1</a>] C. M., V. P., Moore R. <em>et al.</em>, ``_High-Resolution Global Maps of 21st-Century Forest Cover Change_'', Science, vol. 342, number 6160, pp. 850--853, November 2013.

[<a id="cit-Li2017" href="#call-Li2017">2</a>] Li Yan, Sulla-Menashe Damien, Motesharrei Safa <em>et al.</em>, ``_Inconsistent estimates of forest cover change in China between 2000 and 2013 from multiple datasets: differences in parameters, spatial resolution, and definitions_'', Scientific Reports, vol. 7, number 8748, pp. , August 2017.

[<a id="cit-Hansen2013a" href="#call-Hansen2013a">3</a>] C. M., V. P., Moore R. <em>et al.</em>, ``_Supplementary Materials for: High-Resolution Global Maps of 21st-Century Forest Cover Change_'', Sciene, vol. 342, number 6160, pp. 1--32, November 2013.  [online](http://science.sciencemag.org/content/suppl/2013/11/14/342.6160.850.DC1)

[<a id="cit-Tropek2014" href="#call-Tropek2014">4</a>] Tropek Robert, Sedl{\'{a}}{\v{c}}ek Ond{\v{r}}ej, Beck Jan <em>et al.</em>, ``_Comment on High-resolution global maps of 21st-century forest cover change_'', Science, vol. 344, number 981, pp. ,  2014.

[<a id="cit-Bellot2014" href="#call-Bellot2014">5</a>] Bellot Franz-Fabian, Bertram Mathias, Navratilb Peter <em>et al.</em>, ``_The high-resolution global map of 21st-century forest cover change from the University of Maryland (Hansen Map) is hugely overestimating deforestation in Indonesia_'', FORCLIME Press release, vol. , number , pp. ,  2014.  [online](http://www.forclime.org/documents/press_release/FORCLIME_Overestimation%20of%20Deforestation.pdf)

[<a id="cit-Li2017a" href="#call-Li2017a">6</a>] Li Yan, Sulla-Menashe Damien, Motesharrei Safa <em>et al.</em>, ``_Supplementary Information for Inconsistent estimates of forest cover change in China between 2000 and 2013 from multiple datasets_'', Scientific reports, vol. 7, number 8748, pp. , August 2017.

